In [3]:
import pandas as  pd 
import numpy as np 
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
import seaborn as sns # seaborn gets shortened to sns
# We want our plots to appear in the notebook
%matplotlib inline 



In [4]:
# from google.colab import drive
# drive.mount('/content/drive')

In [5]:
# def download(file):
#   #the url and auth_value from the website 
#   url = f"https://api.zindi.africa/v1/competitions/next-gen-wifi-throughput-prediction-challenge/files/{file}"
#   myobj = {'auth_token': "x7nYdKyCrmLZfSkD3GJBxNkQ"} 

#   x = requests.post(url, data = myobj,stream=True)
#   target_path = file

#   handle = open(target_path, "wb")
#   for chunk in x.iter_content(chunk_size=512):
#       if chunk:  # filter out keep-alive new chunks
#           handle.write(chunk)
#   handle.close()

In [6]:
# download('Test.csv')
# download('Train.csv')
# download('TrainTargets.csv')
# download('SampleSubmission.csv')

In [7]:
train = '/kaggle/input/zindihach/Train.csv'
test = '/kaggle/input/zindihach/Test.csv'
target = '/kaggle/input/zindihach/TrainTargets.csv'

In [8]:
df = pd.read_csv('/kaggle/input/zindihach/Train.csv')

In [9]:
target = pd.read_csv('/kaggle/input/zindihach/TrainTargets.csv')

In [10]:
test_df= pd.read_csv('/kaggle/input/zindihach/Test.csv')

In [11]:
def preprocess(data,is_train=True):

  dep=data['Deployment'].unique()
  rows=[]
  for dd in dep:
    df=data[data.Deployment==dd]
    
    toxp_map=dict()
    rssi_map=dict()
    flag=0
    for i, j in df.iterrows():
        if('STA' in j[0]):
          rssi_map[j[0]]={f'AP{i}':j[i] for i in range(1,10)}
        else:
          toxp_map[j[0]]={(j.index[i],'STA'+str(int(j[i]))) for i in range(1,10)}

    
    for exp,elems in toxp_map.items():
      rows1=[]
      for e in elems:
        row=[dd,exp]
        if(e[-1]!='STA0'):
          row.extend([e[0],e[1],rssi_map[e[-1]][e[0]]])
          rows.append(row)
  

      #rows.extend(rows1)
    
  data=pd.DataFrame(rows)
  data.columns = ['Deployment','TXOP','mainAP','mainSTA','mainRSSI']
  data['ID']='Deployment'+data['Deployment'].astype(str)+'_'+data['TXOP']+'_'+data['mainAP']+'_'+data['mainSTA']
  if(is_train):
    
    data=pd.merge(data,target,on='ID')
  
  
  return data
  






In [12]:
train=preprocess(df)
test = preprocess(test_df,False)

In [13]:
train.head()

,Deployment,TXOP,mainAP,mainSTA,mainRSSI,ID,Target
0,9391,TXOP1,AP7,STA7,-63.71,Deployment9391_TXOP1_AP7_STA7,108.0
1,9391,TXOP1,AP1,STA1,-64.87,Deployment9391_TXOP1_AP1_STA1,30.0
2,9391,TXOP1,AP5,STA5,-66.72,Deployment9391_TXOP1_AP5_STA5,54.0
3,9391,TXOP1,AP2,STA2,-67.19,Deployment9391_TXOP1_AP2_STA2,0.0
4,9391,TXOP2,AP6,STA6,-65.12,Deployment9391_TXOP2_AP6_STA6,156.0


In [14]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1749596 entries, 0 to 1749595
Data columns (total 7 columns):
 #   Column      Dtype  
---  ------      -----  
 0   Deployment  int64  
 1   TXOP        object 
 2   mainAP      object 
 3   mainSTA     object 
 4   mainRSSI    float64
 5   ID          object 
 6   Target      float64
dtypes: float64(2), int64(1), object(4)
memory usage: 106.8+ MB


In [15]:
train['mainAP'].unique()

array(['AP7', 'AP1', 'AP5', 'AP2', 'AP6', 'AP3', 'AP8', 'AP9', 'AP4'],
      dtype=object)

In [16]:
train['Deployment'].nunique()

7000

In [17]:
train['mainSTA'].unique()

array(['STA7', 'STA1', 'STA5', 'STA2', 'STA6', 'STA3', 'STA8', 'STA9',
       'STA4', 'STA24', 'STA13', 'STA34', 'STA31', 'STA28', 'STA22',
       'STA29', 'STA27', 'STA32', 'STA35', 'STA17', 'STA20', 'STA30',
       'STA15', 'STA12', 'STA21', 'STA18', 'STA36', 'STA23', 'STA19',
       'STA10', 'STA25', 'STA16', 'STA26', 'STA33', 'STA11', 'STA14',
       'STA43', 'STA44', 'STA37', 'STA39', 'STA45', 'STA40', 'STA41',
       'STA38', 'STA42'], dtype=object)

In [18]:
# # Let's make it look a little prettier
# corr_matrix = train_df.corr()
# plt.figure(figsize=(155, 100))
# sns.heatmap(corr_matrix, 
#             annot=True, 
#             linewidths=0.5, 
#             fmt= ".2f", 
#             cmap="YlGnBu");

In [19]:
def one_hot_encode_columns(df, column_names):
    df_encoded = df.copy()
    encoder = OneHotEncoder(sparse=False)
    
    for column_name in column_names:
        # Extract the column to be encoded
        X = df_encoded[[column_name]]

        # Encode the column
        one_hot_encoded = encoder.fit_transform(X)

        # Create a new DataFrame from the one-hot encoded array
        one_hot_encoded_df = pd.DataFrame(one_hot_encoded, columns=encoder.categories_[0])

        # Replace the original column with the one-hot encoded version
        df_encoded = pd.concat([df_encoded.drop(column_name, axis=1), one_hot_encoded_df], axis=1)
        
    return df_encoded


In [20]:
train_df = one_hot_encode_columns(train, ['TXOP', 'mainAP', 'mainSTA'])

test_df = one_hot_encode_columns(test, ['TXOP', 'mainAP', 'mainSTA'])

In [21]:
train_df.head()

,Deployment,mainRSSI,ID,Target,TXOP1,TXOP10,TXOP100,TXOP11,TXOP12,TXOP13,...,STA41,STA42,STA43,STA44,STA45,STA5,STA6,STA7,STA8,STA9
0,9391,-63.71,Deployment9391_TXOP1_AP7_STA7,108.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,9391,-64.87,Deployment9391_TXOP1_AP1_STA1,30.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,9391,-66.72,Deployment9391_TXOP1_AP5_STA5,54.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,9391,-67.19,Deployment9391_TXOP1_AP2_STA2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,9391,-65.12,Deployment9391_TXOP2_AP6_STA6,156.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [22]:
def add_aggregate_columns(df, column, functions):
    for func in functions:
        if func == 'mean':
            df['mean_' + column] = df[column].mean()
        elif func == 'median':
            df['median_' + column] = df[column].median()
        elif func == 'std':
            df['std_' + column] = df[column].std()
        else:
            raise ValueError('Invalid aggregate function: {}'.format(func))


add_aggregate_columns(train_df, 'mainRSSI', ['mean', 'median', 'std'])
add_aggregate_columns(test_df, 'mainRSSI', [ 'median'])

In [23]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1749596 entries, 0 to 1749595
Columns: 161 entries, Deployment to std_mainRSSI
dtypes: float64(159), int64(1), object(1)
memory usage: 2.1+ GB


In [24]:
train_df1 = train_df.drop(labels=['mean_mainRSSI','std_mainRSSI'], axis=1)

In [25]:
train_df1.head()

,Deployment,mainRSSI,ID,Target,TXOP1,TXOP10,TXOP100,TXOP11,TXOP12,TXOP13,...,STA42,STA43,STA44,STA45,STA5,STA6,STA7,STA8,STA9,median_mainRSSI
0,9391,-63.71,Deployment9391_TXOP1_AP7_STA7,108.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,-64.87
1,9391,-64.87,Deployment9391_TXOP1_AP1_STA1,30.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-64.87
2,9391,-66.72,Deployment9391_TXOP1_AP5_STA5,54.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-64.87
3,9391,-67.19,Deployment9391_TXOP1_AP2_STA2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-64.87
4,9391,-65.12,Deployment9391_TXOP2_AP6_STA6,156.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,-64.87


In [26]:
test_df

,Deployment,mainRSSI,ID,TXOP1,TXOP10,TXOP100,TXOP11,TXOP12,TXOP13,TXOP14,...,STA42,STA43,STA44,STA45,STA5,STA6,STA7,STA8,STA9,median_mainRSSI
0,9373,-69.12,Deployment9373_TXOP1_AP9_STA35,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-64.87
1,9373,-67.19,Deployment9373_TXOP2_AP1_STA3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-64.87
2,9373,-64.87,Deployment9373_TXOP2_AP4_STA13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-64.87
3,9373,-63.71,Deployment9373_TXOP2_AP3_STA9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-64.87
4,9373,-65.58,Deployment9373_TXOP3_AP4_STA14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-64.87
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
749919,5144,-63.71,Deployment5144_TXOP98_AP3_STA5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-64.87
749920,5144,-65.58,Deployment5144_TXOP99_AP9_STA17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-64.87
749921,5144,-63.71,Deployment5144_TXOP99_AP3_STA5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-64.87
749922,5144,-64.61,Deployment5144_TXOP100_AP8_STA16,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-64.87


In [27]:
train_df1.isna().sum()

Deployment         0
mainRSSI           0
ID                 0
Target             0
TXOP1              0
                  ..
STA6               0
STA7               0
STA8               0
STA9               0
median_mainRSSI    0
Length: 159, dtype: int64

In [28]:
test_df.isna().sum()

Deployment         0
mainRSSI           0
ID                 0
TXOP1              0
TXOP10             0
                  ..
STA6               0
STA7               0
STA8               0
STA9               0
median_mainRSSI    0
Length: 158, dtype: int64

In [29]:
train_df1.shape

(1749596, 159)

In [30]:
test_df.shape

(749924, 158)

In [31]:
test_df = test_df.drop(labels='ID', axis=1)

In [32]:
train_df1 = train_df1.drop(labels='ID', axis=1)

In [33]:
train_df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1749596 entries, 0 to 1749595
Columns: 158 entries, Deployment to median_mainRSSI
dtypes: float64(157), int64(1)
memory usage: 2.1 GB


In [34]:
# if train_df1.isnull().values.any():
#   # If there are missing values, we will drop rows with missing values
#   train_df1 = train_df1.dropna()
# else:
#   # If there are no missing values, we will proceed with scaling
#   pass

# # Now, we will initialize the scaler and fit it to the data
# scaler = MinMaxScaler()
# scaler.fit(train_df1)


In [35]:
# import matplotlib.pyplot as plt

# # # Plot the box plots
# # train_df1.plot(kind='box', subplots=True, layout=(158,158), sharex=False, sharey=False, figsize=(10,8))
# # plt.show()

# # If you find any extreme outliers, you can handle them using the following code:

# # For example, to trim the outliers, you can use the following code:
# Q1 = train_df1.quantile(0.25)
# Q3 = train_df1.quantile(0.75)
# IQR = Q3 - Q1

# # Remove the outliers
# train_df1 = train_df1[~((train_df1 < (Q1 - 1.5 * IQR)) |(train_df1 > (Q3 + 1.5 * IQR))).any(axis=1)]


In [36]:
train_df1.head()

,Deployment,mainRSSI,Target,TXOP1,TXOP10,TXOP100,TXOP11,TXOP12,TXOP13,TXOP14,...,STA42,STA43,STA44,STA45,STA5,STA6,STA7,STA8,STA9,median_mainRSSI
0,9391,-63.71,108.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,-64.87
1,9391,-64.87,30.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-64.87
2,9391,-66.72,54.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-64.87
3,9391,-67.19,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-64.87
4,9391,-65.12,156.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,-64.87


In [37]:
from sklearn.preprocessing import MinMaxScaler


# scaler = MinMaxScaler()

# # Fit the scaler to the data
# scaler.fit(train_df1)

# # Transform the data using the scaler
# df_scaled = scaler.transform(train_df1)

# # Load the data
# X = train_df1.drop(['Target'],axis=1) # feature matrix
# y = train_df1['Target'] # target vector

In [40]:

print(train_df1.columns)

Index(['Deployment', 'mainRSSI', 'Target', 'TXOP1', 'TXOP10', 'TXOP100',
       'TXOP11', 'TXOP12', 'TXOP13', 'TXOP14',
       ...
       'STA42', 'STA43', 'STA44', 'STA45', 'STA5', 'STA6', 'STA7', 'STA8',
       'STA9', 'median_mainRSSI'],
      dtype='object', length=158)


In [39]:
import xgboost as xgb
from sklearn.model_selection import KFold

X = train_df1.drop(['Target',axis=1) # feature matrix
y = train_df1['Target'] # target vector

# Create a KFold object with 5 splits
kfold = KFold(n_splits=5, shuffle=True)

# Initialize the model
model = xgb.XGBRegressor()

# Iterate through the splits of the dataset and train the model
for train_index, test_index in kfold.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    model.fit(X_train, y_train)

# Use the trained model to make predictions on the test set



KeyError: "None of [Int64Index([      0,       1,       3,       4,       7,       8,      13,\n                 14,      15,      16,\n            ...\n            1749582, 1749584, 1749585, 1749586, 1749588, 1749589, 1749590,\n            1749591, 1749593, 1749595],\n           dtype='int64', length=1399676)] are in the [columns]"

In [ ]:
# predictions = model.predict(test_df)

In [ ]:
# from lightgbm import LGBMRegressor
# from sklearn.model_selection import KFold
# from sklearn.metrics import mean_squared_error as mse


# folds = KFold(n_splits = 5)

# predictions = []
# losses = []
# X=train[['mainRSSI']]
# y=train['Target']
# X_ts=test[['mainRSSI']]

# for i,( train_index, test_index) in enumerate(folds.split(X, y)):
#   print(50*'--')
#   print(f'Fold {i+1}/5')
#   print(50*'--')
#   X_train, X_test, y_train, y_test = X.iloc[train_index], X.iloc[test_index], y.iloc[train_index], y.iloc[test_index]

#   # Train model
#   model =LGBMRegressor(n_estimators=1000)
#   model.fit(X_train,y_train)
#   # Make predictions
#   preds = model.predict(X_ts)
#   y_pred = model.predict(X_test)
#   # Append predictions and losses
#   predictions.append(preds)
#   loss = mse(y_test, y_pred,squared=False)
#   # Print loss
#   print(f'{i+1}:  {loss}\n')
#   losses.append(loss)
  
# print(f'Mean loss  {np.mean(losses)}')

In [ ]:
# #Making submission
# ss=test[['ID']]
# ss['Target']=np.mean(predictions,0)
# ss

In [ ]:
# ss['Target']=ss['Target'].astype(np.float16)#Otherwise file will be too large

In [ ]:
# ss.to_csv('ss.csv',index=False)

In [ ]:
ss